# Talks markdown generator for academicpages

Adapted from the talks notebook from academicpages

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, date, location, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

Note: edit in Excel and save as tsv. Then open in Notepad and save as with utf-8 encoding.

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [2]:
confs = pd.read_csv("confs.tsv", sep="\t", header=0)
confs

,title,type,url_slug,date,location,description
0,Grace Hopper Celebration of Women in Computing...,NaN,hopper,2014-10-08,"Phoenix, AZ",NaN
1,ICER 2015,NaN,icer,2015-08-10,"Omaha, NE",NaN
2,Women Empowered in STEM (weSTEM) 2017,NaN,weSTEM,2017-01-28,"Urbana, IL",NaN
3,CHI 2017,NaN,chi,2017-05-06,"Denver, CO",NaN
4,Women Empowered in STEM (weSTEM) 2018,NaN,weSTEM,2018-02-10,"Urbana, IL",NaN
5,CSCW 2018,NaN,cscw,2018-11-03,"Jersey City, NJ",NaN
6,Women Empowered in STEM (weSTEM) 2019,NaN,weSTEM,2019-02-23,"Champaign, IL",NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [3]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in confs.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: conferences" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + str(item.type) + '"\n'
    else:
        md += 'type: "Conference"\n'
    
    md += "permalink: /conferences/" + html_filename + "\n"
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
        
    md += 'excerpt: "' 
    
    if len(str(item.description)) > 3:
        md += item.description + "  \n"
    else:
        md += "Attended " + str(item.title) + '"\n'  
           
    md += "---\n"
    
    #start of main text
    
    md += "\n" + str(item.location) + "  \n"         
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_conferences/" + md_filename, 'w') as f:
        f.write(md)